## 用folium生成具有OSM底图的交互式的底图并另存为html文件

In [ ]:
# 用geopandas读取转换后的文件
import folium
import geopandas as gpd
from branca.colormap import linear
import json

import matplotlib.pyplot as plt
import numpy as np
import matplotlib

In [ ]:
# 通过geopandas读取GeoJSON文件以方便查看属性值
gdf = gpd.read_file('../Data/Transport/Cycling/GEOJSON/cycle_parking.geojson')
gdf.shape

In [ ]:
null_count = gdf['PRK_PROVIS'].isnull().sum()

print(f"PRK_PROVIS列中有 {null_count} 个空值")

In [ ]:
# 直接读取geojson文件并在osm底图上进行可视化


# 设置伦敦市中心的经纬度
london_center = [51.5074, -0.1278]

# 创建一个地图对象，中心设为伦敦市中心
m = folium.Map(location=[51.5074, -0.1278], zoom_start=13)
"""
#----------------------------------------------
# 展示连续变量
# 读取 GeoJSON 文件
with open('../Data/Transport/Cycling/GEOJSON/cycle_parking.geojson') as f:
    data = json.load(f)

# 遍历所有要素，为 'PRK_CPT' 属性的空值赋值为 0.0
for feature in data['features']:
    if feature['properties'].get('PRK_CPT') is None:
        feature['properties']['PRK_CPT'] = 0.0
        
# 创建一个线性颜色映射
# 假设连续变量的范围是从 0 到 100
colormap = linear.YlOrRd_09.scale(0, 200)
colormap.caption = '连续变量的颜色映射'
# 将颜色映射图例添加到地图
folium.GeoJson(
    data,
    name='cycle_parking', # 使用文件名作为图层名称
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties'].get('PRK_CPT', 0)),
        'color': 'black',  # 线颜色
        'weight': 2,
        'fillOpacity': 0.7
    }
).add_to(m)


# 将颜色映射图例添加到地图
colormap.add_to(m)
"""
#-------------------------------------------------
# 展示分类变量
# 读取 GeoJSON 文件
with open('../Data/Transport/Cycling/GEOJSON/restricted_route.geojson') as f:
    data = json.load(f)

# 获取所有分类变量的唯一值
categories = set(feature['properties']['BOROUGH'] for feature in data['features'])

# 使用 matplotlib 生成颜色映射
colors = plt.cm.plasma(np.linspace(0, 1, len(categories)))  # 使用 plasma 颜色映射, 也可以使用其他颜色映射
color_map = {category: matplotlib.colors.to_hex(color) for category, color in zip(categories, colors)}

# 添加 GeoJSON 图层到地图，根据分类变量自动分配颜色
folium.GeoJson(
    data,
    name='geojson',
    style_function=lambda feature: {
        'fillColor': matplotlib.colors.to_hex(color_map[feature['properties']['BOROUGH']]),
        'color': 'black',  # 边界颜色
        'weight': 2,
        'fillOpacity': 0.5
    }
).add_to(m)


# 添加图层控制器
folium.LayerControl().add_to(m)


#------------------------------------------
# 在Jupyter Notebook中直接显示地图
#m
# 保存为html文件
m.save('../Data/Transport/Cycling/cycling_map.html')

## Air Monitoring Sites

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point

In [2]:
df = pd.read_csv("../Data/AirQuality/MonitoringSiteSpecies.csv")

In [4]:

df = df[df['Latitude']!=0]
df = df[df['Longitude']!=0]

In [5]:
df['DateMeasurementFinished'] = pd.to_datetime(df['DateMeasurementFinished'], errors='coerce')
df['DateMeasurementStarted'] = pd.to_datetime(df['DateMeasurementStarted'], errors='coerce')

# set some conditions for filtering
# condition1: 'DateClosed' is NAN or later than'2017-12-16 00:00'
# condition2: 'DateOpened' is earlier than '2018-03-17 00:00'
condition = (df['DateMeasurementFinished'].isna() | (df['DateMeasurementFinished'] > pd.Timestamp('2018-03-17 00:00'))) & (df['DateMeasurementStarted'] < pd.Timestamp('2017-12-16 00:00'))

# apply the filter condition to the dataframe
filtered_df = df[condition]

In [6]:
filtered_df.sample(7)

,LocalAuthorityCode,LocalAuthorityName,SiteCode,SiteName,SiteType,DateClosed,DateOpened,Latitude,Longitude,LatitudeWGS84,LongitudeWGS84,DataOwner,DataManager,SiteLink,SpeciesCode,SpeciesDescription,DateMeasurementStarted,DateMeasurementFinished
120,7,City of London,CT3,City of London - The Aldgate School,Urban Background,2023/6/30 0:00,2003/1/1 0:00,51.513847,-0.077766,6712695.644,-8656.836094,City of London,King's College London,http://www.londonair.org.uk/london/asp/publicd...,PM10,PM10 Particulate,2015-01-01 00:00:00,2023-06-30
208,11,Greenwich,GN3,Greenwich - Plumstead High Street,Roadside,NaN,2006/1/27 0:00,51.486957,0.095111,6707887.038,10587.708090,Greenwich,King's College London,http://www.londonair.org.uk/london/asp/publicd...,NO2,Nitrogen Dioxide,2008-01-01 00:00:00,NaT
104,6,Camden,CD1,Camden - Swiss Cottage,Kerbside,NaN,1996/4/16 13:30,51.544219,-0.175284,6718130.265,-19512.525620,Camden,King's College London,http://www.londonair.org.uk/london/asp/publicd...,NO2,Nitrogen Dioxide,2001-12-10 17:00:00,NaT
23,3,Bexley,BQ7,Bexley - Belvedere West,Urban Background,NaN,2010/8/24 0:00,51.494649,0.137279,6709278.288,14951.936430,Bexley,King's College London,http://www.londonair.org.uk/london/asp/publicd...,O3,Ozone,2010-08-24 00:00:00,NaT
460,27,Richmond,RHG,Richmond Upon Thames - Chertsey Road,Roadside,2020/8/6 0:00,2016/5/24 0:00,51.453142,-0.341218,6701844.177,-37984.229380,Richmond,King's College London,http://www.londonair.org.uk/london/asp/publicd...,NO2,Nitrogen Dioxide,2016-05-24 11:15:00,2020-08-06
492,29,Sutton,ST8,Sutton - Beddington Lane,Industrial,2020/10/16 0:00,2012/4/4 0:00,51.383565,-0.136418,6689424.488,-15185.966870,Sutton,King's College London,http://www.londonair.org.uk/london/asp/publicd...,NO2,Nitrogen Dioxide,2001-10-08 09:00:00,2020-07-30
200,11,Greenwich,GN4,Greenwich - Fiveways Sidcup Rd A20,Roadside,NaN,2011/1/24 0:00,51.434663,0.064222,6702412.549,4535.536335,Greenwich,King's College London,http://www.londonair.org.uk/london/asp/publicd...,NO2,Nitrogen Dioxide,2011-01-24 00:00:00,NaT


In [7]:
geometry = [Point(xy) for xy in zip(filtered_df['Longitude'], filtered_df['Latitude'])]

# 创建GeoDataFrame
gdf = gpd.GeoDataFrame(filtered_df, geometry=geometry)

In [8]:
gdf['geometry']

5       POINT (0.17789 51.56375)
6       POINT (0.17789 51.56375)
7       POINT (0.13286 51.52939)
8       POINT (0.13286 51.52939)
18      POINT (0.15891 51.49061)
                 ...            
604    POINT (-0.15459 51.52254)
605    POINT (-0.15459 51.52254)
606    POINT (-0.15279 51.51393)
607    POINT (-0.15279 51.51393)
612    POINT (-0.11671 51.51197)
Name: geometry, Length: 207, dtype: geometry

In [9]:
import folium

In [14]:
# 设置坐标参考系统(CRS)为英国国家网格参考系统 (EPSG:27700)，并转换为WGS84 (EPSG:4326)
gdf.crs = 'epsg:4326'
gdf = gdf.to_crs(epsg=4326)

# 创建 folium 地图对象
m = folium.Map(
    location=[gdf.geometry.y.mean(), gdf.geometry.x.mean()], # 设置地图的中心点
    zoom_start=10 # 初始缩放级别
)

# 在地图上添加点，同时检查几何对象是否为空
for _, row in gdf.iterrows():
    if row.geometry is not None and not row.geometry.is_empty:
        # 使用内置的图标
        icon = folium.Icon(
            icon='glyphicon glyphicon-map-marker',  # 使用内置的glyphicon图标，例如地图标记
            prefix='glyphicon',  # 指定图标集，这里使用Bootstrap的glyphicon
            color='red',  # 指定图标颜色
        )

        """
        # use the customized icon
        # 定义一个自定义图标
        icon_url = 'https://github.com/BohaoSuCC/DeepLearning-Explore/blob/main/Advanced-Models/pin.png'  
        icon = folium.CustomIcon(
            icon_url,
            icon_size=(30, 30)  # 指定图标的大小，(宽度, 高度) 单位是像素
        )"""
        

        popup_content = f"{row['SiteCode']}<br>{row['SpeciesCode']}" 
        folium.Marker(
            location=[row.geometry.y, row.geometry.x],
            icon=icon,
            popup=popup_content  
        ).add_to(m)


folium.LayerControl().add_to(m)


m.save('../Data/AirQuality/SitesDistribution.html')

## Congestion Scoot Monitoring Sites

In [18]:
df2 = pd.read_csv("../Data/CongestionScoot/Metadata(siteCoordinates.csv")

In [19]:
df2.sample(5)

,ID,Easting,Northing
3713,27-188,512764,180223
2614,16-114,548333,188074
3794,28-091,518555,185237
1754,09-213,530445,173615
1706,09-132,530682,179660


In [21]:

gdf2 = gpd.GeoDataFrame(
    df2, 
    geometry=[Point(xy) for xy in zip(df2.Easting, df2.Northing)]
)
# set the initial CRS to EPSG:27700 (British National Grid)
# 设置初始CRS为EPSG:27700（英国国家网格参考系统）
gdf2.crs = 'epsg:27700'
# convert the CRS to EPSG:4326
# 转换CRS到EPSG:4326
gdf2 = gdf2.to_crs(epsg=4326)

In [23]:

# create a merged map
# 创建 folium 地图对象
m_merged = folium.Map(
    location=[gdf.geometry.y.mean(), gdf.geometry.x.mean()], 
    zoom_start=10 
)

# plot the first layer
# 在地图上添加点，同时检查几何对象是否为空
for _, row in gdf.iterrows():
    if row.geometry is not None and not row.geometry.is_empty:
        # 使用内置的图标
        icon = folium.Icon(
            icon='glyphicon glyphicon-map-marker',  
            prefix='glyphicon',  
            color='red',  
        )

        """
        # use the customized icon
        # 定义一个自定义图标
        icon_url = 'https://github.com/BohaoSuCC/DeepLearning-Explore/blob/main/Advanced-Models/pin.png'  
        icon = folium.CustomIcon(
            icon_url,
            icon_size=(30, 30)  # 指定图标的大小，(宽度, 高度) 单位是像素
        )"""
        

        popup_content = f"{row['SiteCode']}<br>{row['SpeciesCode']}" 
        folium.Marker(
            location=[row.geometry.y, row.geometry.x],
            icon=icon,
            popup=popup_content  
        ).add_to(m_merged)


for _, row in gdf2.iterrows():
    if row.geometry is not None and not row.geometry.is_empty:
        # 使用内置的图标
        icon = folium.Icon(
            icon='glyphicon glyphicon-map-marker',  
            prefix='glyphicon',  
            color='blue',  
        )

        """
        # use the customized icon
        # 定义一个自定义图标
        icon_url = 'https://github.com/BohaoSuCC/DeepLearning-Explore/blob/main/Advanced-Models/pin.png'  
        icon = folium.CustomIcon(
            icon_url,
            icon_size=(30, 30)  # 指定图标的大小，(宽度, 高度) 单位是像素
        )"""
        

        popup_content = f"{row['ID']}" 
        folium.Marker(
            location=[row.geometry.y, row.geometry.x],
            icon=icon,
            popup=popup_content  
        ).add_to(m_merged)

# add the layer control
# 添加图层切换控件
folium.LayerControl().add_to(m_merged)


m_merged.save('../Data/AirQuality/AQ_Congestion_merged.html')

In [32]:
# after checking the map, the ID=02-064 and ID=02-144 are close to the air quality site SiteCode=CD9
# so import those three datasets and analyze the correlation between the air quality and the congestion


AQ_CD9 = pd.read_csv("../Data/AirQuality/PM25/CD9.csv")
CS_02_064 = pd.read_csv("../Data/CongestionScoot/MergedCSVs/02-064.csv")
CS_02_144 = pd.read_csv("../Data/CongestionScoot/MergedCSVs/02-144.csv")

In [33]:
AQ_CD9.sample(4)

,MeasurementDateGMT,Camden - Euston Road: PM2.5 Particulate (ug/m3)
901,2018-01-23 13:00,6.6
1929,2018-03-07 09:00,14.6
1987,2018-03-09 19:00,22.9
124,2017-12-22 04:00,15.1


In [34]:
CS_02_064.sample(4)

,DateTime,Date,Time,SatMean,SatBand,FlowMean
7507,43164.197917,5-Mar-2018,04:45,21.67,0-79%,995
4155,43129.270833,29-Jan-2018,06:30,47.67,0-79%,2561
7421,43163.302083,4-Mar-2018,07:15,36.33,0-79%,1514
2286,43109.802083,9-Jan-2018,19:15,210.67,>= 100%,2436


In [35]:
CS_02_144.sample(4)

,DateTime,Date,Time,SatMean,SatBand,FlowMean
2643,43113.520833,13-Jan-2018,12:30,70.00,0-79%,2539
1846,43105.218750,5-Jan-2018,05:15,25.67,0-79%,1029
7914,43168.437500,9-Mar-2018,10:30,74.00,0-79%,3037
2520,43112.239583,12-Jan-2018,05:45,45.67,0-79%,1563


处理数据，合并两个表格

In [42]:
# merge the air quality and congestion datasets
# 将Date和Time列合并为一个字符串列
CS_02_064['DateTimeStr'] = CS_02_064['Date'] + ' ' + CS_02_064['Time']

# convert the merged string to datetime format and create a new column
# 将合并后的字符串转换为datetime格式，并创建一个新列
CS_02_064['MeasurementDateGMT'] = pd.to_datetime(CS_02_064['DateTimeStr'], format='%d-%b-%Y %H:%M')

# delete the middle DateTimeStr column 
# 删除中间的DateTimeStr列
CS_02_064.drop(columns=['DateTimeStr'], inplace=True)

AQ_CD9['MeasurementDateGMT'] = pd.to_datetime(AQ_CD9['MeasurementDateGMT'])
CS_02_064['MeasurementDateGMT'] = pd.to_datetime(CS_02_064['MeasurementDateGMT'])

CS_02_064.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8640 entries, 0 to 8639
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DateTime            8640 non-null   float64       
 1   Date                8640 non-null   object        
 2   Time                8640 non-null   object        
 3   SatMean             8640 non-null   float64       
 4   SatBand             8640 non-null   object        
 5   FlowMean            8640 non-null   int64         
 6   MeasurementDateGMT  8640 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 472.6+ KB


In [43]:
Merged = pd.merge(AQ_CD9, CS_02_064, on='MeasurementDateGMT', how='left')

In [44]:
Merged.sample(14)

,MeasurementDateGMT,Camden - Euston Road: PM2.5 Particulate (ug/m3),DateTime,Date,Time,SatMean,SatBand,FlowMean
944,2018-01-25 08:00:00,13.4,43125.333333,25-Jan-2018,08:00,59.67,0-79%,2830.0
1830,2018-03-03 06:00:00,41.0,43162.250000,3-Mar-2018,06:00,25.67,0-79%,1137.0
1219,2018-02-05 19:00:00,19.7,43136.791667,5-Feb-2018,19:00,54.67,0-79%,2662.0
1186,2018-02-04 10:00:00,7.2,43135.416667,4-Feb-2018,10:00,56.33,0-79%,2541.0
2049,2018-03-12 09:00:00,19.5,43171.375000,12-Mar-2018,09:00,51.00,0-79%,2828.0
513,2018-01-07 09:00:00,6.1,43107.375000,7-Jan-2018,09:00,43.33,0-79%,1722.0
1417,2018-02-14 01:00:00,NaN,43145.041667,14-Feb-2018,01:00,29.00,0-79%,1242.0
1235,2018-02-06 11:00:00,15.7,43137.458333,6-Feb-2018,11:00,72.00,0-79%,2697.0
1816,2018-03-02 16:00:00,32.5,43161.666667,2-Mar-2018,16:00,56.67,0-79%,2520.0
1252,2018-02-07 04:00:00,4.2,43138.166667,7-Feb-2018,04:00,18.00,0-79%,790.0
